In [1]:
import torch
import boda

import sys
sys.path.insert(0, "/home1/smaruj/AkitaMini-pytorch")  # Add the directory where "ledidi" is located
from model import SeqNN

In [2]:
from akita_helper import plot_map, from_upper_triu

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


In [4]:
model = SeqNN()

# Load the saved model weights
model.load_state_dict(torch.load('/home1/smaruj/AkitaMini-pytorch/best_model.pth'))

/tmp/SLURM_209730/ipykernel_2031170/2425661507.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/home1/smaruj/AkitaMini-pytorch/best_mod

<All keys matched successfully>

In [5]:
model = model.to(device)

In [6]:
device = next(model.parameters()).device
print(device)

cuda:0


In [7]:
# Set the model to evaluation mode (important for inference)
model.eval()

SeqNN(
  (stochastic_reverse_complement): StochasticReverseComplement()
  (stochastic_shift): StochasticShift()
  (re_lu): ReLU()
  (conv_block_1): ConvBlock(
    (conv): Conv1d(4, 96, kernel_size=(11,), stride=(1,), padding=(5,), bias=False)
    (batch_norm): BatchNorm1d(96, eps=0.001, momentum=0.0735, affine=True, track_running_stats=True)
    (pool): MaxPool1d(kernel_size=8, stride=8, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_tower): ConvTower(
    (conv_tower): Sequential(
      (0): ReLU()
      (1): Conv1d(96, 96, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (2): BatchNorm1d(96, eps=0.001, momentum=0.0735, affine=True, track_running_stats=True)
      (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): ReLU()
      (5): Conv1d(96, 96, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (6): BatchNorm1d(96, eps=0.001, momentum=0.0735, affine=True, track_running_stats=True)
      (7): MaxPool1d(kernel_size

In [8]:
from torchinfo import summary

summary(model, input_size=(2, 4, 32768), col_names=["output_size", "num_params"])

Layer (type:depth-idx)                   Output Shape              Param #
SeqNN                                    [2, 1, 1953]              --
├─StochasticReverseComplement: 1-1       [2, 4, 32768]             --
├─StochasticShift: 1-2                   [2, 4, 32768]             --
├─ReLU: 1-3                              [2, 4, 32768]             --
├─ConvBlock: 1-4                         [2, 96, 4096]             --
│    └─Conv1d: 2-1                       [2, 96, 32768]            4,224
│    └─BatchNorm1d: 2-2                  [2, 96, 32768]            192
│    └─MaxPool1d: 2-3                    [2, 96, 4096]             --
├─ConvTower: 1-5                         [2, 96, 64]               --
│    └─Sequential: 2-4                   [2, 96, 64]               --
│    │    └─ReLU: 3-1                    [2, 96, 4096]             --
│    │    └─Conv1d: 3-2                  [2, 96, 4096]             46,080
│    │    └─BatchNorm1d: 3-3             [2, 96, 4096]             192
│    │

In [9]:
# Load the test data
test_data = torch.load('/home1/smaruj/AkitaMini-pytorch/test_data.pt')

# Assume test_data is a tuple of (X, y)
X_test, y_test = test_data

/tmp/SLURM_209730/ipykernel_2031170/413687493.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_data = torch.load('/home1/smaruj/AkitaMini-pytorch/test_data.pt')


In [10]:
num_examples = 3
X_batch = X_test[:num_examples]
y_batch = y_test[:num_examples]

In [11]:
X_batch, y_batch = X_batch.to(device), y_batch.to(device)

In [12]:
# Get predictions
with torch.no_grad():
    predictions = model(X_batch)

In [13]:
predictions.shape

torch.Size([3, 1, 1953])

In [14]:
import numpy as np

# Function to get the indices of the upper triangular part for a given matrix size
def get_upper_triu_indices(dim, num_diags=2):
    return np.triu_indices(dim, k=num_diags)

# The matrix size you are working with
dim = 64
# The starting and ending indices for the chunk (0:21, 21:38)
start_row, end_row = 0, 21
start_col, end_col = 21, 38

# Get the full upper triangular indices for the 64x64 matrix
full_indices = get_upper_triu_indices(dim)

# Now create a mask to extract the relevant slice of the matrix
mask = ((full_indices[0] >= start_row) & (full_indices[0] < end_row) &
        (full_indices[1] >= start_col) & (full_indices[1] < end_col))

# Extract the corresponding indices from the full 64x64 vector
sub_indices_in_full_vector = np.where(mask)[0]



In [19]:
# Function to convert a list of indices into a list of slices
def indices_to_slices(indices):
    slices = []
    start = indices[0]
    
    for i in range(1, len(indices)):
        if indices[i] != indices[i-1] + 1:  # Check if the indices are contiguous
            slices.append(slice(start, indices[i-1] + 1))
            start = indices[i]
    
    # Append the last slice, without including step=None
    slices.append(slice(start, indices[-1] + 1))
    
    # Remove the step argument from the slices (if it's None)
    return [slice(s.start, s.stop) for s in slices]


In [20]:
slices = indices_to_slices(sub_indices_in_full_vector)

In [24]:
slices

[slice(19, 36, None),
 slice(80, 97, None),
 slice(140, 157, None),
 slice(199, 216, None),
 slice(257, 274, None),
 slice(314, 331, None),
 slice(370, 387, None),
 slice(425, 442, None),
 slice(479, 496, None),
 slice(532, 549, None),
 slice(584, 601, None),
 slice(635, 652, None),
 slice(685, 702, None),
 slice(734, 751, None),
 slice(782, 799, None),
 slice(829, 846, None),
 slice(875, 892, None),
 slice(920, 937, None),
 slice(964, 981, None),
 slice(1007, 1024, None),
 slice(1050, 1066, None)]

# CODA

In [ ]:
# Fast SeqProp

In [25]:
logits = torch.randn(
    1, 4, 336
)

left_flank = X_batch[1,:, :10328].unsqueeze(0)

right_flank = X_batch[1,:, :-10664].unsqueeze(0)

In [26]:
left_flank.shape, right_flank.shape

(torch.Size([1, 4, 10328]), torch.Size([1, 4, 22104]))

In [ ]:
10328 + 22104 + 336

In [27]:
params = boda.generator.StraightThroughParameters(
    data=logits, left_flank=left_flank, right_flank=right_flank, 
    n_samples=1, use_affine=False
)
params.cuda()

StraightThroughParameters(
  (norm): InstanceNorm1d(4, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
)

In [ ]:
# energy = boda.generator.MinGapEnergy(model, target_feature=21, target_alpha=-1, a_min=-2., a_max=6.)
# energy = boda.generator.MinGapEnergy(model, target_feature=slice(96,980), target_alpha=-1, a_min=-2., a_max=6.)
energy = boda.generator.MinGapEnergy(model, target_feature=slice(96,980), target_alpha=-1, a_min=-2., a_max=6.)

#Higher or Lower?:
# - If you want the target feature to be higher in the energy calculation (i.e., the energy should be lower when the target feature is higher), 
# then you’ll use a positive target_alpha.
# - If you want the target feature to be lower (i.e., the energy should be lower when the target feature is lower), 
# then you’ll use a negative target_alpha.

In [ ]:
FSP = boda.generator.FastSeqProp(energy, params)

In [ ]:
FSP.run(n_steps=1000, lr_scheduler=True, create_plot=True)
proposals = params.get_sample()
proposals.shape

In [ ]:
squeezed_proposals = proposals.squeeze(0)

In [ ]:
squeezed_proposals.shape, logits.shape

In [ ]:
og_seq = X_batch[1,:,:]

In [ ]:
og_seq.shape

In [ ]:
og_seq[:, 10328:10664] = squeezed_proposals

In [ ]:
og_seq_dim = og_seq.unsqueeze(0)

In [ ]:
with torch.no_grad():
    predictions_new = model(og_seq_dim)

In [ ]:
predictions_new.shape

In [ ]:
plot_map(from_upper_triu(predictions[1], matrix_len=64, num_diags=2), vmin=-0.6, vmax=0.6, palette="RdBu_r", width=5, height=5)

In [ ]:
plot_map(from_upper_triu(predictions_new, matrix_len=64, num_diags=2), vmin=-0.6, vmax=0.6, palette="RdBu_r", width=5, height=5)

In [ ]:
map_one = from_upper_triu(predictions[1], matrix_len=64, num_diags=2)
map_two = from_upper_triu(predictions_new, matrix_len=64, num_diags=2)

In [ ]:
np.nanmean(map_one), np.nanmean(map_two)